In [33]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

raw_text = '''
A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret!
The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word.
His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy.
the barber went up a huge mountain.'''

# 문장 토큰화
sentences = sent_tokenize(raw_text)
print(sentences)

vocab = {}
preprocessed_sentences = []
stop_words = set(stopwords.words('english'))

# 단어 토큰화
for sentence in sentences:
    tokenized_sentence = word_tokenize(sentence)
    result = []
    for word in tokenized_sentence: 
        word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄인다.
        if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거한다.
            if len(word) > 2: # 단어 길이가 2이하인 경우에 대하여 추가로 단어를 제거한다.
                result.append(word)
                if word not in vocab:
                    vocab[word] = 0 
                vocab[word] += 1
    preprocessed_sentences.append(result) 
print(preprocessed_sentences)

tokenizer = Tokenizer()

# fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성.
tokenizer.fit_on_texts(preprocessed_sentences) 

print(tokenizer.word_index) # 각 단어의 빈도수가 높은 순서대로 인덱스가 부여

print(tokenizer.word_counts) # 각 단어별 사용된 카운트를 보여줌

# 입력으로 들어온 코퍼스에 대해 각 단어를 정해진 인덱스로 변환
print(tokenizer.texts_to_sequences(preprocessed_sentences))

# 숫자 0과 OOV를 고려해서 단어 집합의 크기는 +2
vocab_size = 5 # 상위 5개 단어만 사용
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token = 'OOV')
tokenizer.fit_on_texts(preprocessed_sentences)
print('없는 단어를 위한 OOV 인덱스 : {}'.format(tokenizer.word_index['OOV']))

encoded = tokenizer.texts_to_sequences(preprocessed_sentences)
print(encoded)

max_len = max(len(item) for item in encoded)
print('가장 길이가 긴 문장 길이 :',max_len)

encodedWithPadding = pad_sequences(encoded, padding='post', truncating='post', maxlen=5)
print(encodedWithPadding)
print(encodedWithPadding.shape)

['\nA barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']
[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}
OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept'

In [24]:
# 단어 집합(Vocabulary) = 텍스트의 모든 단어를 중복을 허용하지 않고 모아놓은것
# 원-핫 인코딩 = 단어 집합의 크기를 차원으로 하고,
# 표현하고 싶은 단어의 인덱스에 1의 값을 부여하고,
# 다른 인덱스에는 0을 부여하는 단어의 벡터 표현 (One-Hot vector)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

text = "나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
print('단어 집합 :',tokenizer.word_index)

encoded = tokenizer.texts_to_sequences([text])[0]
print(encoded)
print('문장 길이 :',len(encoded))

one_hot = to_categorical(encoded)
print(one_hot)
print(one_hot.shape)

단어 집합 : {'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}
[4, 2, 5, 1, 2, 6, 3, 1, 1, 3, 7]
문장 길이 : 11
[[0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
(11, 8)


In [38]:
values = [['당신에게 드리는 마지막 혜택!', 1],
['내일 뵐 수 있을지 확인 부탁드...', 0],
['도연씨. 잘 지내시죠? 오랜만입...', 0],
['(광고) AI로 주가를 예측할 수 있다!', 1]]
columns = ['메일 본문', '스팸 메일 유무']

df = pd.DataFrame(values, columns=columns)
print(df)

x = df['메일 본문']
y = df['스팸 메일 유무']

print('x 데이터 :',x.to_list())
print('y 데이터 :',y.to_list())

# 8:2의 비율로 훈련 데이터와 테스트 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

print('x 훈련 데이터 :')
print(x_train)
print('x 테스트 데이터 :')
print(x_test)

                    메일 본문  스팸 메일 유무
0        당신에게 드리는 마지막 혜택!         1
1    내일 뵐 수 있을지 확인 부탁드...         0
2    도연씨. 잘 지내시죠? 오랜만입...         0
3  (광고) AI로 주가를 예측할 수 있다!         1
x 데이터 : ['당신에게 드리는 마지막 혜택!', '내일 뵐 수 있을지 확인 부탁드...', '도연씨. 잘 지내시죠? 오랜만입...', '(광고) AI로 주가를 예측할 수 있다!']
y 데이터 : [1, 0, 0, 1]
x 훈련 데이터 :
1      내일 뵐 수 있을지 확인 부탁드...
2      도연씨. 잘 지내시죠? 오랜만입...
3    (광고) AI로 주가를 예측할 수 있다!
Name: 메일 본문, dtype: object
x 테스트 데이터 :
0    당신에게 드리는 마지막 혜택!
Name: 메일 본문, dtype: object
